<a href="https://colab.research.google.com/github/sairagillani18k/AirBnb-Zillow-Price_Prediciton/blob/master/LR_7030_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sympy import *
import matplotlib.pyplot as plt
import operator

from IPython.core.display import display

import torch
from torch.autograd import Variable
import torch.utils.data as data_utils
import torch.nn.init as init

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection

init_printing(use_unicode=True)

In [ ]:
data = pd.read_csv("Desktop/data.csv")

In [ ]:
x = data.drop(["id", "diagnosis", "Unnamed: 32"], axis=1)
diag = { "M": 1, "B": 0}
y = data["diagnosis"].replace(diag)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)

In [ ]:
def create_model(layer_dims):
    model = torch.nn.Sequential()
    for idx, dim in enumerate(layer_dims):
        if (idx < len(layer_dims) - 1):
            module = torch.nn.Linear(dim, layer_dims[idx + 1])
            init.xavier_normal(module.weight)
            model.add_module("linear" + str(idx), module)
        else:
            model.add_module("sig" + str(idx), torch.nn.Sigmoid())
        if (idx < len(layer_dims) - 2):
            model.add_module("relu" + str(idx), torch.nn.ReLU())

    return model

In [ ]:
from sklearn import model_selection

iter = 100
accuracy = list()

for i in range(iter): # No of Iteration
    print("iteration No. ", i)
    count = 0
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=100)
    scaler = StandardScaler()
    transformed = scaler.fit_transform(x_train)

    train = data_utils.TensorDataset(torch.from_numpy(transformed).float(),
                                     torch.from_numpy(y_train.to_numpy()).float())
    dataloader = data_utils.DataLoader(train, batch_size=128, shuffle=False)
    
    scaler = StandardScaler()
    transformed = scaler.fit_transform(x_test)

    test_set = torch.from_numpy(transformed).float()
    test_valid = torch.from_numpy(y_test.to_numpy()).float()
    
    ## Create model and hyper parameters
    dim_in = x_train.shape[1]
    dim_out = 1

    layer_dims = [dim_in, 20, 10, dim_out]

    model = create_model(layer_dims)

    loss_fn = torch.nn.MSELoss(size_average=False)
    learning_rate = 0.0007
    n_epochs = 10
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    ## Now run model
    history = { "loss": [], "accuracy": [], "loss_val": [], "accuracy_val": [] }
    for epoch in range(n_epochs):
        loss = None

        for idx, (minibatch, target) in enumerate(dataloader):
            y_pred = model(Variable(minibatch))

            loss = loss_fn(y_pred, Variable(target.float()))
            prediction = [1 if x > 0.5 else 0 for x in y_pred.data.numpy()]
            correct = (prediction == target.numpy()).sum()

            # This can be uncommented for a per mini batch feedback
            #history["loss"].append(loss.data[0])
            #history["accuracy"].append(100 * correct / len(prediction))

            y_val_pred = model(Variable(test_set))
            loss_val = loss_fn(y_val_pred, Variable(test_valid.float()))
            prediction_val = [1 if x > 0.5 else 0 for x in y_val_pred.data.numpy()]
            correct_val = (prediction_val == test_valid.numpy()).sum()

            # This can be uncommented for a per mini batch feedback
            #history["loss_val"].append(loss_val.data[0])
            #history["accuracy_val"].append(100 * correct_val / len(prediction_val))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        history["loss"].append(loss.data)
        history["accuracy"].append(100 * correct / len(prediction))
        history["loss_val"].append(loss_val.data)
        history["accuracy_val"].append(100 * correct_val / len(prediction_val))

        print("Loss, accuracy, val loss, val acc at epoch", epoch + 1,history["loss"][-1], 
              history["accuracy"][-1], history["loss_val"][-1], history["accuracy_val"][-1] )


    index, value = max(enumerate(history["accuracy_val"]), key=operator.itemgetter(1))

    print("Best accuracy was {} at iteration {}".format(value, index))
    accuracy.append(value)
    print("accuracy is ", value)

f = open("cross_validation.txt", "a+")
f.write("average cv accuracy for {0} iterations and {1} folds is ".format(iter,count) + str(np.sum(np.array(accuracy))/(iter*count)) + '\n')
f.write("standard deviation cv accuracy for {0} iterations and {1} folds is ".format(iter,count) + str(np.std(np.array(accuracy))) + '\n')
f.close()

iteration No.  0


<ipython-input-9-25c249d58b9c>:6: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(module.weight)
D:\ananconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
D:\ananconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\ananconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([171])) that is different to the input size (torch.Size([171, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, tar

Loss, accuracy, val loss, val acc at epoch 1 tensor(56.1617) 7.142857142857143 tensor(7995.7905) 26.31578947368421
Loss, accuracy, val loss, val acc at epoch 2 tensor(54.9425) 7.142857142857143 tensor(7879.7695) 24.56140350877193
Loss, accuracy, val loss, val acc at epoch 3 tensor(53.8826) 7.142857142857143 tensor(7776.3237) 22.22222222222222
Loss, accuracy, val loss, val acc at epoch 4 tensor(52.8916) 7.142857142857143 tensor(7683.2251) 22.22222222222222
Loss, accuracy, val loss, val acc at epoch 5 tensor(51.9404) 7.142857142857143 tensor(7599.6694) 22.80701754385965
Loss, accuracy, val loss, val acc at epoch 6 tensor(50.9744) 7.142857142857143 tensor(7524.8481) 23.976608187134502
Loss, accuracy, val loss, val acc at epoch 7 tensor(50.0820) 14.285714285714286 tensor(7458.9487) 26.900584795321638
Loss, accuracy, val loss, val acc at epoch 8 tensor(49.2519) 14.285714285714286 tensor(7400.9609) 33.91812865497076
Loss, accuracy, val loss, val acc at epoch 9 tensor(48.4722) 21.428571428571

<ipython-input-9-25c249d58b9c>:6: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(module.weight)
D:\ananconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
D:\ananconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\ananconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([171])) that is different to the input size (torch.Size([171, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, tar

 52.046783625730995
Loss, accuracy, val loss, val acc at epoch 3 tensor(53.3015) 42.857142857142854 tensor(8008.3687) 53.801169590643276
Loss, accuracy, val loss, val acc at epoch 4 tensor(51.2811) 50.0 tensor(7850.3955) 59.64912280701754
Loss, accuracy, val loss, val acc at epoch 5 tensor(49.6344) 50.0 tensor(7728.2441) 57.89473684210526
Loss, accuracy, val loss, val acc at epoch 6 tensor(48.3201) 50.0 tensor(7636.8296) 55.55555555555556
Loss, accuracy, val loss, val acc at epoch 7 tensor(47.2405) 42.857142857142854 tensor(7567.2861) 57.30994152046784
Loss, accuracy, val loss, val acc at epoch 8 tensor(46.4019) 50.0 tensor(7513.0552) 57.30994152046784
Loss, accuracy, val loss, val acc at epoch 9 tensor(45.7392) 57.142857142857146 tensor(7471.3237) 55.55555555555556
Loss, accuracy, val loss, val acc at epoch 10 tensor(45.2108) 57.142857142857146 tensor(7439.6914) 54.97076023391813
Best accuracy was 59.64912280701754 at iteration 3
accuracy is  59.64912280701754
iteration No.  3
Loss, a

...

<ipython-input-9-25c249d58b9c>:6: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(module.weight)
D:\ananconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
D:\ananconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\ananconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([171])) that is different to the input size (torch.Size([171, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, tar

 6 tensor(42.3507) 71.42857142857143 tensor(7294.3516) 61.40350877192982
Loss, accuracy, val loss, val acc at epoch 7 tensor(42.4008) 71.42857142857143 tensor(7259.7520) 61.40350877192982
Loss, accuracy, val loss, val acc at epoch 8 tensor(42.4232) 71.42857142857143 tensor(7233.4375) 61.40350877192982
Loss, accuracy, val loss, val acc at epoch 9 tensor(42.4314) 71.42857142857143 tensor(7212.4810) 61.40350877192982
Loss, accuracy, val loss, val acc at epoch 10 tensor(42.4300) 71.42857142857143 tensor(7197.6343) 61.40350877192982
Best accuracy was 61.40350877192982 at iteration 3
accuracy is  61.40350877192982
iteration No.  97
Loss, accuracy, val loss, val acc at epoch 1 tensor(46.2643) 78.57142857142857 tensor(7425.8979) 73.6842105263158
Loss, accuracy, val loss, val acc at epoch 2 tensor(45.4613) 71.42857142857143 tensor(7371.1680) 71.34502923976608
Loss, accuracy, val loss, val acc at epoch 3 tensor(44.8071) 71.42857142857143 tensor(7326.0054) 69.5906432748538
Loss, accuracy, val los

<ipython-input-9-25c249d58b9c>:6: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(module.weight)
D:\ananconda3\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
D:\ananconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\ananconda3\lib\site-packages\torch\nn\modules\loss.py:445: UserWarning: Using a target size (torch.Size([171])) that is different to the input size (torch.Size([171, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, tar

tensor(48.5003) 42.857142857142854 tensor(7510.7988) 41.52046783625731
Loss, accuracy, val loss, val acc at epoch 9 tensor(47.0879) 42.857142857142854 tensor(7419.4502) 46.78362573099415
Loss, accuracy, val loss, val acc at epoch 10 tensor(45.9229) 50.0 tensor(7355.7559) 53.21637426900585
Best accuracy was 53.21637426900585 at iteration 9
accuracy is  53.21637426900585
iteration No.  99
Loss, accuracy, val loss, val acc at epoch 1 tensor(51.4024) 21.428571428571427 tensor(7523.2622) 37.42690058479532
Loss, accuracy, val loss, val acc at epoch 2 tensor(50.0950) 35.714285714285715 tensor(7451.5601) 38.59649122807018
Loss, accuracy, val loss, val acc at epoch 3 tensor(48.9459) 64.28571428571429 tensor(7398.9224) 41.52046783625731
Loss, accuracy, val loss, val acc at epoch 4 tensor(47.9720) 64.28571428571429 tensor(7361.2402) 45.02923976608187
Loss, accuracy, val loss, val acc at epoch 5 tensor(47.1414) 71.42857142857143 tensor(7336.3696) 46.78362573099415
Loss, accuracy, val loss, val acc

<ipython-input-10-b4bbf517b735>:82: RuntimeWarning: divide by zero encountered in double_scalars
  f.write("average cv accuracy for {0} iterations and {1} folds is ".format(iter,count) + str(np.sum(np.array(accuracy))/(iter*count)) + '\n')
